# Stage 5:(Dashboard)
## Goals
* The final stage aims a developing a simple interactive dashboard based on the analysis you have done so far. In this we will be utilizing Plotly (https://plotly.com/) along with Dash (https://plotly.com/dash/) as our framework.

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from datetime import date

from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

C:\Users\Manish\AppData\Local\Temp\ipykernel_268\3722043857.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Manish\AppData\Local\Temp\ipykernel_268\3722043857.py:7: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


### Loading the dataset

In [2]:
data = pd.read_csv('../../../../data/stage_1/superdataset.csv')
data = data[data.countyFIPS != 0]

### Adding columns of daily cases and deaths

In [3]:
data['confirmed_daily'] = data['Confirmed'].diff()
data['deaths_daily'] = data['Deaths'].diff()

### Here we are saving the data for the map development in the last task

In [4]:
# saving the data to new dataframe for later use in USA map generation
data_for_map = data

### Adding columns of normalized data

In [5]:
data['confirmed_daily_normalized'] = (data['confirmed_daily']/data['population'])*100000
data['deaths_daily_normalized'] = (data['deaths_daily']/data['population'])*100000

### print the data with added columns

In [6]:
data

,countyFIPS,County_Name,State,Date,Confirmed,Deaths,population,confirmed_daily,deaths_daily,confirmed_daily_normalized,deaths_daily_normalized
748,1001,autauga county,AL,2020-01-22,0,0,55869,NaN,NaN,NaN,NaN
749,1001,autauga county,AL,2020-01-23,0,0,55869,0.0,0.0,0.000000,0.0
750,1001,autauga county,AL,2020-01-24,0,0,55869,0.0,0.0,0.000000,0.0
751,1001,autauga county,AL,2020-01-25,0,0,55869,0.0,0.0,0.000000,0.0
752,1001,autauga county,AL,2020-01-26,0,0,55869,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2346471,56045,weston county,WY,2022-02-03,1491,17,6927,6.0,0.0,86.617583,0.0
2346472,56045,weston county,WY,2022-02-04,1496,17,6927,5.0,0.0,72.181319,0.0
2346473,56045,weston county,WY,2022-02-05,1496,17,6927,0.0,0.0,0.000000,0.0
2346474,56045,weston county,WY,2022-02-06,1496,17,6927,0.0,0.0,0.000000,0.0


### Converting the date column into datetime from string data type

In [7]:
#convert dates to datetime() object
data['Date'] = data.Date.apply(lambda d: datetime.strptime(d, '%Y-%m-%d'))

### Here we are saving the data for extracting the state in the further task

In [8]:
state_data = data

### Grouping the data by Date and summing all data

In [9]:
data = data.groupby(["Date"]).sum().reset_index()
data

,Date,countyFIPS,Confirmed,Deaths,population,confirmed_daily,deaths_daily,confirmed_daily_normalized,deaths_daily_normalized
0,2020-01-22,93245330,712,1,321279906,-73031676.0,-848118.0,-4.098812e+08,-4.497241e+06
1,2020-01-23,93245330,723,1,321279906,11.0,0.0,1.774044e+00,0.000000e+00
2,2020-01-24,93245330,730,1,321279906,7.0,0.0,1.845313e-01,0.000000e+00
3,2020-01-25,93245330,740,1,321279906,10.0,0.0,2.552533e+00,0.000000e+00
4,2020-01-26,93245330,748,1,321279906,8.0,0.0,5.282089e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...
743,2022-02-03,93245330,72283911,841597,321279906,274783.0,2073.0,4.073291e+05,2.547721e+03
744,2022-02-04,93245330,72684661,845515,321279906,400750.0,3918.0,3.458894e+05,4.090329e+03
745,2022-02-05,93245330,72733724,845746,321279906,49063.0,231.0,4.767555e+04,1.638108e+02
746,2022-02-06,93245330,72777948,846024,321279906,44224.0,278.0,3.891743e+04,1.892202e+02


### Removing the first row which contains negative values

In [10]:
data = data.iloc[1: , :]
data

,Date,countyFIPS,Confirmed,Deaths,population,confirmed_daily,deaths_daily,confirmed_daily_normalized,deaths_daily_normalized
1,2020-01-23,93245330,723,1,321279906,11.0,0.0,1.774044,0.000000
2,2020-01-24,93245330,730,1,321279906,7.0,0.0,0.184531,0.000000
3,2020-01-25,93245330,740,1,321279906,10.0,0.0,2.552533,0.000000
4,2020-01-26,93245330,748,1,321279906,8.0,0.0,5.282089,0.000000
5,2020-01-27,93245330,758,1,321279906,10.0,0.0,0.855012,0.000000
...,...,...,...,...,...,...,...,...,...
743,2022-02-03,93245330,72283911,841597,321279906,274783.0,2073.0,407329.099719,2547.721431
744,2022-02-04,93245330,72684661,845515,321279906,400750.0,3918.0,345889.351831,4090.328933
745,2022-02-05,93245330,72733724,845746,321279906,49063.0,231.0,47675.546582,163.810790
746,2022-02-06,93245330,72777948,846024,321279906,44224.0,278.0,38917.429734,189.220165


### Selecting only the required columns

In [11]:
data1 = data[['Date','confirmed_daily','deaths_daily','confirmed_daily_normalized','deaths_daily_normalized']]
data1

,Date,confirmed_daily,deaths_daily,confirmed_daily_normalized,deaths_daily_normalized
1,2020-01-23,11.0,0.0,1.774044,0.000000
2,2020-01-24,7.0,0.0,0.184531,0.000000
3,2020-01-25,10.0,0.0,2.552533,0.000000
4,2020-01-26,8.0,0.0,5.282089,0.000000
5,2020-01-27,10.0,0.0,0.855012,0.000000
...,...,...,...,...,...
743,2022-02-03,274783.0,2073.0,407329.099719,2547.721431
744,2022-02-04,400750.0,3918.0,345889.351831,4090.328933
745,2022-02-05,49063.0,231.0,47675.546582,163.810790
746,2022-02-06,44224.0,278.0,38917.429734,189.220165


## Main Graph: 
### Allow for selection of date to show the trend of COVID-19 cases and deaths.

This is the link we referred to plot cases and deaths in dash 
* https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e

We have referred the following links to add date filter
* https://github.com/Coding-with-Adam/Dash-by-Plotly/blob/master/Dash%20Components/DatePickerRange/datepicker.py
* https://dash.plotly.com/dash-core-components/datepickerrange

## Cases and Death selection with Dates

In [12]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Trend of COVID-19 Cases & Deaths"),
    dcc.Graph(id='graph'),

    dcc.DatePickerRange(
        id='my-date-picker-range',
        min_date_allowed=date(2020, 1, 23),
        max_date_allowed=date(2022, 2, 7),
        initial_visible_month=date(2020, 2, 1),
        start_date=date(2020, 1, 23),
        end_date=date(2022, 1, 20)
    ),
    html.Div(id='output-container-date-picker-range'),
       html.Br(),
       html.Br(),
       html.Br(),
    dcc.RadioItems(
        id = 'cases-deaths',
        options=[{'label': 'Cases', 'value': 'confirmed_daily'},{'label': 'Deaths', 'value': 'deaths_daily'}],
        value='confirmed_daily',    
),   
    
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    #Output('output-container-date-picker-range', 'children'),
    [Input('my-date-picker-range', 'start_date'),
    Input('my-date-picker-range', 'end_date'), 
    Input('cases-deaths', 'value')]
)
def update_figure(start_date, end_date,value):

    dff = data1[data1['Date']>=start_date] 
    dff2= data1[data1['Date']<=end_date]
    z = pd.merge(dff,dff2)
    #dff2
    if value == 'confirmed_daily':
        fig = px.scatter(z, x='Date', y="confirmed_daily", color="confirmed_daily"
                     ,render_mode="webgl", title="Daily Cases")
        return fig
    elif value == 'deaths_daily':
        fig = px.scatter(z, x='Date', y="deaths_daily", color="deaths_daily"
                     ,render_mode="webgl", title="Daily Deaths")
        return fig

# Run app and display result inline in the notebook
app.run_server(mode='inline',port=10)

## Main Graph:

* Allow for linear or log mode selection on the number of cases and deaths.
* Incorporate your best model prediction trend line - Linear / Non-Linear.

In [13]:
data1

,Date,confirmed_daily,deaths_daily,confirmed_daily_normalized,deaths_daily_normalized
1,2020-01-23,11.0,0.0,1.774044,0.000000
2,2020-01-24,7.0,0.0,0.184531,0.000000
3,2020-01-25,10.0,0.0,2.552533,0.000000
4,2020-01-26,8.0,0.0,5.282089,0.000000
5,2020-01-27,10.0,0.0,0.855012,0.000000
...,...,...,...,...,...
743,2022-02-03,274783.0,2073.0,407329.099719,2547.721431
744,2022-02-04,400750.0,3918.0,345889.351831,4090.328933
745,2022-02-05,49063.0,231.0,47675.546582,163.810790
746,2022-02-06,44224.0,278.0,38917.429734,189.220165


### Adding column of logarithemic cases and deaths

In [14]:
#Find the log normalized vales
data1['log_cases'] = np.log(data1['confirmed_daily'])
data1['log_deaths'] = np.log(data1['deaths_daily'])

In [15]:
data1

,Date,confirmed_daily,deaths_daily,confirmed_daily_normalized,deaths_daily_normalized,log_cases,log_deaths
1,2020-01-23,11.0,0.0,1.774044,0.000000,2.397895,-inf
2,2020-01-24,7.0,0.0,0.184531,0.000000,1.945910,-inf
3,2020-01-25,10.0,0.0,2.552533,0.000000,2.302585,-inf
4,2020-01-26,8.0,0.0,5.282089,0.000000,2.079442,-inf
5,2020-01-27,10.0,0.0,0.855012,0.000000,2.302585,-inf
...,...,...,...,...,...,...,...
743,2022-02-03,274783.0,2073.0,407329.099719,2547.721431,12.523737,7.636752
744,2022-02-04,400750.0,3918.0,345889.351831,4090.328933,12.901093,8.273337
745,2022-02-05,49063.0,231.0,47675.546582,163.810790,10.800860,5.442418
746,2022-02-06,44224.0,278.0,38917.429734,189.220165,10.697023,5.627621


We have referred to the below link for plotting graphs with radio buttons:
* https://dash.plotly.com/dash-core-components/radioitems
* https://www.youtube.com/watch?v=O4Xi_i4otII&list=PLH6mU1kedUy8fCzkTTJlwsf2EnV_UvOV-&index=18
* https://www.youtube.com/watch?v=FuJOsZgo4nU

In [16]:
app = JupyterDash(__name__)
app.layout = html.Div([
html.H1("Trend of COVID-19 Cases Linear/Log"),
dcc.Graph(id='graph'),
dcc.RadioItems(
id = 'linear-log',
#options=['confirmed_daily', 'log_cases','deaths_daily','log_deaths'],
    options=[{'label': 'Cases', 'value': 'confirmed_daily'},{'label': 'Deaths', 'value': 'deaths_daily'},
                 {'label': 'Log Cases', 'value': 'log_cases'},{'label': 'Log Deaths', 'value': 'log_deaths'}],
value='confirmed_daily'
),
html.Br(),

dcc.RadioItems(
id = 'linear-nonlinear',
#options=['Linear Trendline','Non Linear Trendline':'value'],
options=[{'label': 'Linear Trendline', 'value': 'Linear'},{'label': 'Non Linear Trendline', 'value': 'Non Linear'}],
value='Linear'
),
html.Br(),

dcc.DatePickerRange(
id='my-date-picker-range',
min_date_allowed=date(2020, 1, 23),
max_date_allowed=date(2022, 2, 7),
initial_visible_month=date(2020, 2, 1),
start_date=date(2020, 1, 23),
end_date=date(2022, 1, 20)
),
html.Div(id='output-container-date-picker-range'),
])

# Define callback to update graph
@app.callback(
Output('graph', 'figure'),
[Input('my-date-picker-range', 'start_date'),
Input('my-date-picker-range', 'end_date'),
Input('linear-log', 'value'),
Input('linear-nonlinear', 'value')
]
)


def update_figure(start_date, end_date,value,value1):
    dff = data1[data1['Date']>=start_date]
    dff2= data1[data1['Date']<=end_date]
    z = pd.merge(dff,dff2)

    if value1 == 'Linear':
        fig = px.scatter(z, x='Date', y=value, color=value
        ,render_mode="webgl", title="Daily Cases/Log Cases", trendline="ols",trendline_color_override="red")
        return fig
    
    elif value1 == 'Non Linear':
        fig = px.scatter(z, x='Date', y=value, color=value
        ,render_mode="webgl", title="Daily Cases/Log Cases", trendline="expanding",trendline_color_override="red")
        return fig

# Run app and display result inline in the notebook
app.run_server(mode='inline',port= 40)

## Main Graph:

* Plot the trend line using moving average. Use 7-day moving average.
* Allow for selection of multiple states on the same graph.

### Grouping the state data by State and Date, summing the columns

In [17]:
state_data = state_data.groupby(["State","Date"]).sum().reset_index()
state_data = state_data.iloc[1:]
state_data

,State,Date,countyFIPS,Confirmed,Deaths,population,confirmed_daily,deaths_daily,confirmed_daily_normalized,deaths_daily_normalized
1,AK,2020-01-23,48940,0,0,709319,0.0,0.0,0.000000,0.0
2,AK,2020-01-24,48940,0,0,709319,0.0,0.0,0.000000,0.0
3,AK,2020-01-25,48940,0,0,709319,0.0,0.0,0.000000,0.0
4,AK,2020-01-26,48940,0,0,709319,0.0,0.0,0.000000,0.0
5,AK,2020-01-27,48940,0,0,709319,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
37395,WY,2022-02-03,1288529,149126,1650,578759,744.0,0.0,3426.408271,0.0
37396,WY,2022-02-04,1288529,149842,1650,578759,716.0,0.0,2928.265791,0.0
37397,WY,2022-02-05,1288529,149842,1650,578759,0.0,0.0,0.000000,0.0
37398,WY,2022-02-06,1288529,149842,1650,578759,0.0,0.0,0.000000,0.0


### Adding columns of logarithemic cases and deaths

In [18]:
state_data['log_cases'] = np.log(state_data['confirmed_daily'])
state_data['log_deaths'] = np.log(state_data['deaths_daily'])
state_data

,State,Date,countyFIPS,Confirmed,Deaths,population,confirmed_daily,deaths_daily,confirmed_daily_normalized,deaths_daily_normalized,log_cases,log_deaths
1,AK,2020-01-23,48940,0,0,709319,0.0,0.0,0.000000,0.0,-inf,-inf
2,AK,2020-01-24,48940,0,0,709319,0.0,0.0,0.000000,0.0,-inf,-inf
3,AK,2020-01-25,48940,0,0,709319,0.0,0.0,0.000000,0.0,-inf,-inf
4,AK,2020-01-26,48940,0,0,709319,0.0,0.0,0.000000,0.0,-inf,-inf
5,AK,2020-01-27,48940,0,0,709319,0.0,0.0,0.000000,0.0,-inf,-inf
...,...,...,...,...,...,...,...,...,...,...,...,...
37395,WY,2022-02-03,1288529,149126,1650,578759,744.0,0.0,3426.408271,0.0,6.612041,-inf
37396,WY,2022-02-04,1288529,149842,1650,578759,716.0,0.0,2928.265791,0.0,6.573680,-inf
37397,WY,2022-02-05,1288529,149842,1650,578759,0.0,0.0,0.000000,0.0,-inf,-inf
37398,WY,2022-02-06,1288529,149842,1650,578759,0.0,0.0,0.000000,0.0,-inf,-inf


We have referred the below link to create dropdown
* https://dash.plotly.com/dash-core-components/dropdown


We have referred this link to print values of State in dropdown
* https://stackoverflow.com/questions/55436475/graph-not-updating-when-dropdown-value-selected

### Here, we are extracting all the states names using the unique() function 

In [19]:
#selecting all states present in united states
l1 = state_data['State'].unique()
l1

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
       'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI',
       'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV',
       'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
       'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

In [20]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Trend of COVID-19 Cases/Deaths Moving Averages by State"),
    dcc.Graph(id='graph'),
    
    html.Div([
    dcc.Dropdown(l1, 'ND', id='demo-dropdown'),
    html.Div(id='dd-output-container')
    ]),   
    html.Br(),

    dcc.RadioItems(
        id = 'cases_deaths',
        options=[{'label': 'Cases', 'value': 'confirmed_daily'},{'label': 'Deaths', 'value': 'deaths_daily'},
                 {'label': 'Log Cases', 'value': 'log_cases'},{'label': 'Log Deaths', 'value': 'log_deaths'}],
        
        #options=['confirmed_daily','deaths_daily','log_cases','log_deaths'],
        value='confirmed_daily'
),
    html.Br(),
 
    dcc.DatePickerRange(
        id='my-date-picker-range',
        min_date_allowed=date(2020, 1, 23),
        max_date_allowed=date(2022, 2, 7),
        initial_visible_month=date(2020, 2, 1),
        start_date=date(2020, 1, 23),
        end_date=date(2022, 1, 20)
    ),
    html.Div(id='output-container-date-picker-range'),
    
    
    
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input('my-date-picker-range', 'start_date'),
    Input('my-date-picker-range', 'end_date'),
    Input('cases_deaths', 'value'),
     Input('demo-dropdown', 'value')]
)


def update_figure(start_date, end_date,value,value1):

    dff = state_data[state_data['Date']>=start_date] 
    dff2= state_data[state_data['Date']<=end_date]
    z = pd.merge(dff,dff2)
    #print(value1)
    z1 = z[z['State'] == value1]
    fig = px.scatter(z1, x='Date', y=value, color=value
                      ,render_mode="webgl", title="Daily Cases/ Deaths by State",trendline="rolling",trendline_options=dict(window=7),trendline_color_override="red")
    return fig

# Run app and display result inline in the notebook
app.run_server(mode='inline',port=82)

## Map:

* Show US map of cases or deaths (normalized by population). Entire US with county level information. Use mean of county population to normalize for cases and deaths.



### Loading the dataset

In [21]:
data_map = pd.read_csv('../../../../data/stage_1/superdataset.csv')

### Removing state unallocated data

In [22]:
data_map = data_map[data_map['County_Name']!='statewide unallocated']
data_map

,countyFIPS,County_Name,State,Date,Confirmed,Deaths,population
748,1001,autauga county,AL,2020-01-22,0,0,55869
749,1001,autauga county,AL,2020-01-23,0,0,55869
750,1001,autauga county,AL,2020-01-24,0,0,55869
751,1001,autauga county,AL,2020-01-25,0,0,55869
752,1001,autauga county,AL,2020-01-26,0,0,55869
...,...,...,...,...,...,...,...
2346471,56045,weston county,WY,2022-02-03,1491,17,6927
2346472,56045,weston county,WY,2022-02-04,1496,17,6927
2346473,56045,weston county,WY,2022-02-05,1496,17,6927
2346474,56045,weston county,WY,2022-02-06,1496,17,6927


### Grouping the data by County Name and State

In [24]:
data_map = data_map.groupby(['County_Name','countyFIPS','State']).mean().reset_index()

### Calling the dataset generated in the steps above

In [25]:
# Calling data_for_map dataframe
data_for_map

,countyFIPS,County_Name,State,Date,Confirmed,Deaths,population,confirmed_daily,deaths_daily,confirmed_daily_normalized,deaths_daily_normalized
748,1001,autauga county,AL,2020-01-22,0,0,55869,NaN,NaN,NaN,NaN
749,1001,autauga county,AL,2020-01-23,0,0,55869,0.0,0.0,0.000000,0.0
750,1001,autauga county,AL,2020-01-24,0,0,55869,0.0,0.0,0.000000,0.0
751,1001,autauga county,AL,2020-01-25,0,0,55869,0.0,0.0,0.000000,0.0
752,1001,autauga county,AL,2020-01-26,0,0,55869,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2346471,56045,weston county,WY,2022-02-03,1491,17,6927,6.0,0.0,86.617583,0.0
2346472,56045,weston county,WY,2022-02-04,1496,17,6927,5.0,0.0,72.181319,0.0
2346473,56045,weston county,WY,2022-02-05,1496,17,6927,0.0,0.0,0.000000,0.0
2346474,56045,weston county,WY,2022-02-06,1496,17,6927,0.0,0.0,0.000000,0.0


In [26]:
data_map

,County_Name,countyFIPS,State,Confirmed,Deaths,population
0,abbeville county,45001,SC,1821.177807,26.185829,24527.0
1,acadia parish,22001,LA,5321.426471,141.029412,62045.0
2,accomack county,51001,VA,2202.244652,35.042781,32316.0
3,ada county,16001,ID,38200.962567,353.946524,481587.0
4,adair county,19001,IA,564.848930,18.721925,7152.0
...,...,...,...,...,...,...
3082,yuma county,4027,AZ,24952.181818,543.005348,213787.0
3083,yuma county,8125,CO,610.474599,9.891711,10019.0
3084,zapata county,48505,TX,1143.832888,22.903743,14179.0
3085,zavala county,48507,TX,1255.816845,29.292781,11840.0


### Normalizing the data using mean of county population

In [27]:
#Normalization of Data using mean of population
data_map['confirmed_daily_normalized'] = (data_for_map['confirmed_daily']/data_map['population'])*100000
data_map['deaths_daily_normalized'] = (data_for_map['deaths_daily']/data_map['population'])*100000
data_map

,County_Name,countyFIPS,State,Confirmed,Deaths,population,confirmed_daily_normalized,deaths_daily_normalized
0,abbeville county,45001,SC,1821.177807,26.185829,24527.0,NaN,NaN
1,acadia parish,22001,LA,5321.426471,141.029412,62045.0,NaN,NaN
2,accomack county,51001,VA,2202.244652,35.042781,32316.0,NaN,NaN
3,ada county,16001,ID,38200.962567,353.946524,481587.0,NaN,NaN
4,adair county,19001,IA,564.848930,18.721925,7152.0,NaN,NaN
...,...,...,...,...,...,...,...,...
3082,yuma county,4027,AZ,24952.181818,543.005348,213787.0,0.000000,0.0
3083,yuma county,8125,CO,610.474599,9.891711,10019.0,9.981036,0.0
3084,zapata county,48505,TX,1143.832888,22.903743,14179.0,0.000000,0.0
3085,zavala county,48507,TX,1255.816845,29.292781,11840.0,8.445946,0.0


### Checking and removing the NAN values in normalized data column

In [28]:
# Checking for NAN values in cases and deaths
print(data_map['confirmed_daily_normalized'].isna().sum())
print(data_map['deaths_daily_normalized'].isna().sum())

749
749


In [29]:
# Replacing NAN values with 0.
data_map['confirmed_daily_normalized'] = data_map['confirmed_daily_normalized'].fillna(0) 
data_map['deaths_daily_normalized'] = data_map['deaths_daily_normalized'].fillna(0) 

In [30]:
# Checking for NAN again
print(data_map['confirmed_daily_normalized'].isna().sum())
print(data_map['deaths_daily_normalized'].isna().sum())

0
0


In [31]:
data_map

,County_Name,countyFIPS,State,Confirmed,Deaths,population,confirmed_daily_normalized,deaths_daily_normalized
0,abbeville county,45001,SC,1821.177807,26.185829,24527.0,0.000000,0.0
1,acadia parish,22001,LA,5321.426471,141.029412,62045.0,0.000000,0.0
2,accomack county,51001,VA,2202.244652,35.042781,32316.0,0.000000,0.0
3,ada county,16001,ID,38200.962567,353.946524,481587.0,0.000000,0.0
4,adair county,19001,IA,564.848930,18.721925,7152.0,0.000000,0.0
...,...,...,...,...,...,...,...,...
3082,yuma county,4027,AZ,24952.181818,543.005348,213787.0,0.000000,0.0
3083,yuma county,8125,CO,610.474599,9.891711,10019.0,9.981036,0.0
3084,zapata county,48505,TX,1143.832888,22.903743,14179.0,0.000000,0.0
3085,zavala county,48507,TX,1255.816845,29.292781,11840.0,8.445946,0.0


We have referred the following links to create US map of cases or deaths

* https://plotly.com/python/choropleth-maps/

* https://plotly.com/python/builtin-colorscales/

In [32]:
from urllib.request import urlopen
import json

In [33]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("United States Map: Cases and Deaths"),
    dcc.Graph(id="graph"),
    dcc.RadioItems(
        id = 'cases_deaths',
        options=[{'label': 'Cases', 'value': 'confirmed_daily'},{'label': 'Deaths', 'value': 'deaths_daily'}],
        value='confirmed_daily'
),    
])
# Define callback to update graph
@app.callback(
    Output("graph", "figure"),
    Input('cases_deaths', 'value')
)


def update_figure(value):
    
    with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
        counties = json.load(response)
    if value == 'confirmed_daily':
            fig = px.choropleth(data_map, geojson=counties, locations='countyFIPS', color='confirmed_daily_normalized',
                           color_continuous_scale="BuPu",
                           range_color=(0, 12),
                           scope="usa",
                           hover_data=["County_Name", "confirmed_daily_normalized", "deaths_daily_normalized"],
                           labels={'cases':'cases_rate'}
                          )
            fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
            return fig
    elif value == 'deaths_daily':
        fig = px.choropleth(data_map, geojson=counties, locations='countyFIPS', color='deaths_daily_normalized',
                           color_continuous_scale="BuPu",
                           range_color=(0, 12),
                           scope="usa",
                           hover_data=["County_Name", "confirmed_daily_normalized", "deaths_daily_normalized"],
                           labels={'deaths':'deaths_rate'}
                          )
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        return fig
    
    
# Run app and display result inline in the notebook
app.run_server(mode='inline',port=85)